# Marketing Analytics: Data Enrichment Pipeline

## Overview
This notebook documents the ETL (Extract, Transform, Load) process for the Marketing Analytics dashboard. 
We are transforming a Star Schema (Fact + Dimensions) into a single wide Master Table for Tableau.

## Note
This process would have been more appropriate for SQL or even directly with Tableau for the purposes of the produced synthdata. This was just meant as practice with pandas and Jupyter documentation :)

### Data Architecture
* **Fact Table**: `fact_performance.csv` (Transactions)
* **Dimensions**: 
    * `dim_campaign.csv` (Campaign Context)
    * `dim_source.csv` (Channel/Platform Context)
    * `dim_date.csv` (Time Context)

In [2]:
import pandas as pd
import numpy as np

## 1. Data Extraction
Loading the raw CSV files into pandas DataFrames.

In [3]:
# Load the datasets
fact_performance = pd.read_csv('../docs/fact_performance.csv')
dim_campaign = pd.read_csv('../docs/dim_campaign.csv')
dim_source = pd.read_csv('../docs/dim_source.csv')
dim_date = pd.read_csv('../docs/dim_date.csv')

# Verification
print(f"Fact Table Size: {fact_performance.shape}")
print(f"Campaign Dimension Size: {dim_campaign.shape}")
print(f"Source Dimension Size: {dim_source.shape}")
print(f"Date Dimension Size: {dim_date.shape}")

Fact Table Size: (16416, 8)
Campaign Dimension Size: (16, 4)
Source Dimension Size: (8, 3)
Date Dimension Size: (456, 7)


## 2. Data Transformation: Merging
Joining the Fact table with Dimensions to create a unified view.

* **Campaign Info**: Joined on `campaign_id`
* **Source Info**: Joined on `source_id`
* **Date Info**: Joined on `date_key`

In [4]:
# Left join fact table with dimensions
df_master = fact_performance.merge(dim_campaign, on='campaign_id', how='left')
df_master = df_master.merge(dim_source, on='source_id', how='left')
df_master = df_master.merge(dim_date, on='date_key', how='left')

## 3. Feature Engineering: KPI Calculation
Calculating standard marketing metrics. We use `np.where` to handle potential division-by-zero errors safely.

### Formulas
* **CPM (Cost Per Mille)**: $\frac{\text{Spend}}{\text{Impressions}} \times 1000$
* **CTR (Click-Through Rate)**: $\frac{\text{Clicks}}{\text{Impressions}} \times 100$
* **CPC (Cost Per Click)**: $\frac{\text{Spend}}{\text{Clicks}}$
* **Conversion Rate**: $\frac{\text{Conversions}}{\text{Impressions}} \times 100$

In [5]:
# CPM: (Spend / Impressions) * 1000
df_master['CPM'] = np.where(df_master['impressions'] > 0, 
                            (df_master['spend'] / df_master['impressions']) * 1000, 
                            0)

# CTR: (Clicks / Impressions) * 100
df_master['CTR'] = np.where(df_master['impressions'] > 0, 
                            (df_master['clicks'] / df_master['impressions']) * 100, 
                            0)

# CPC: Spend / Clicks
df_master['CPC'] = np.where(df_master['clicks'] > 0, 
                            df_master['spend'] / df_master['clicks'], 
                            0)

# Conversion Rate (Conversions / Impressions * 100)
df_master['Conversion_Rate'] = np.where(df_master['impressions'] > 0, 
                                        (df_master['conversions'] / df_master['impressions']) * 100, 
                                        0)

## 4. Quality Assurance & Export
Previewing the final dataset before exporting to CSV for visualization tools (Tableau/PowerBI).

In [6]:
# Check for nulls or anomalies
print(df_master.info())

# Preview key metrics
df_master[['date', 'campaign_name', 'channel', 'spend', 'CPM', 'CTR', 'CPC', 'Conversion_Rate']].head()

<class 'pandas.DataFrame'>
RangeIndex: 16416 entries, 0 to 16415
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date_key         16416 non-null  int64  
 1   source_id        16416 non-null  int64  
 2   campaign_id      16416 non-null  int64  
 3   impressions      16416 non-null  int64  
 4   clicks           16416 non-null  int64  
 5   spend            16416 non-null  float64
 6   conversions      16416 non-null  int64  
 7   video_views      16416 non-null  int64  
 8   campaign_name    16416 non-null  str    
 9   ad_set_name      16416 non-null  str    
 10  objective        16416 non-null  str    
 11  source_name      16416 non-null  str    
 12  channel          16416 non-null  str    
 13  date             16416 non-null  str    
 14  year             16416 non-null  int64  
 15  month            16416 non-null  int64  
 16  month_name       16416 non-null  str    
 17  quarter          16416 

,date,campaign_name,channel,spend,CPM,CTR,CPC,Conversion_Rate
0,2023-01-01,Business-focused zero tolerance,Programmatic,9.14,1.996069,0.524132,0.380833,0.065516
1,2023-01-02,Business-focused zero tolerance,Programmatic,46.61,3.143801,0.593552,0.529659,0.033725
2,2023-01-03,Business-focused zero tolerance,Programmatic,70.30,4.308126,0.667974,0.644954,0.030641
3,2023-01-04,Business-focused zero tolerance,Programmatic,26.31,2.725860,0.372980,0.730833,0.031082
4,2023-01-05,Business-focused zero tolerance,Programmatic,38.80,6.066291,0.687930,0.881818,0.046904


In [7]:
# Export to CSV
df_master.to_csv('../docs/marketing_analytics_master.csv', index=False)
print("Export complete: ../docs/marketing_analytics_master.csv")

Export complete: ../docs/marketing_analytics_master.csv
